In [ ]:
### 数据太大，无法上传
## 地址是：http://evexdb.org/pmresources/vec-space-models/wikipedia-pubmed-and-PMC-w2v.bin

In [14]:
import os 
import re
import csv
import codecs 
import numpy as np
import pandas as pd

In [15]:
import nltk

In [16]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from gensim.models import KeyedVectors

In [17]:
import keras
print(keras.__version__)  

2.2.4


In [18]:
import tensorflow 
print(tensorflow.__version__) 

1.12.0


In [20]:
EMBEDDING_FILE = 'wikipedia-pubmed-and-PMC-w2v.bin'
print('Indexing word vectors')
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,binary=True)
print('found %s word vectors of words2vec'% len(word2vec.vocab) ) 

Indexing word vectors
found 5443656 word vectors of words2vec


In [21]:
import copy
from keras.preprocessing.sequence import pad_sequences

In [22]:
TEXT_FILE =  'DRUG-AE.rel'

In [23]:
input_data_ae = []
op_labels_ae = []

sentences = [] 

In [30]:
# f = open(TEXT_FILE, 'r')

# for each_line in f.readlines():
#     sent_list = np.zeros([0,200])
#     labels = np.zeros([0,3])
#     tokens = each_line.split("|")
#     sent = tokens[1]
#     if sent in sentences:
#         continue
#     sentences.append(sent)
#     begin_offset = int(tokens[3])
#     end_offset = int(tokens[4])
#     mid_offset = range(begin_offset+1, end_offset)
#     word_tokens = nltk.word_tokenize(sent)
#     offset = 0
#     for each_token in word_tokens:
#         offset = sent.find(each_token, offset)
#         offset1 = copy.deepcopy(offset)
#         offset += len(each_token)
#         if each_token in punctuation or re.search(r'\d', each_token):
#             continue
#         each_token = each_token.lower()
#         each_token = re.sub("[^A-Za-z\-]+","", each_token)
#         if each_token in word2vec.vocab:
#             new_word = word2vec.word_vec(each_token)
#         if offset1 == begin_offset:
#             sent_list = np.append(sent_list, np.array([new_word]), axis=0)
#             labels = np.append(labels, np.array([[0,0,1]]), axis=0)
#         elif offset == end_offset or offset in mid_offset:
#             sent_list = np.append(sent_list, np.array([new_word]), axis=0)
#             labels = np.append(labels, np.array([[0,1,0]]), axis=0)
#         else:
#             sent_list = np.append(sent_list, np.array([new_word]), axis=0)
#             labels = np.append(labels, np.array([[1,0,0]]), axis=0)

#     input_data_ae.append(sent_list)
#     op_labels_ae.append(labels)
# input_data_ae = np.array(input_data_ae)
# op_labels_ae  = np.array(op_labels_ae)

In [34]:
f = open(TEXT_FILE, 'r')

for each_line in f.readlines():
    sent_list = np.zeros([0,200])
    labels = np.zeros([0,3])
    tokens = each_line.split("|")
    sent = tokens[1]
    if sent in sentences:
        continue
    sentences.append(sent)
    begin_offset = int(tokens[3])
    end_offset = int(tokens[4])
    mid_offset = range(begin_offset+1, end_offset)
    word_tokens = nltk.word_tokenize(sent)
    offset = 0
    for each_token in word_tokens:
        offset = sent.find(each_token, offset)
        offset1 = copy.deepcopy(offset)
        offset += len(each_token)
        if each_token in punctuation or re.search(r'\d', each_token):
            continue
        each_token = each_token.lower()
        each_token = re.sub("[^A-Za-z\-]+","", each_token)
        if each_token in word2vec.vocab:
            new_word = word2vec.word_vec(each_token)
        if offset1 == begin_offset:
            sent_list = np.append(sent_list, np.array([new_word]), axis=0)
            labels = np.append(labels, np.array([[0,0,1]]), axis=0)
        elif offset == end_offset or offset in mid_offset:
            sent_list = np.append(sent_list, np.array([new_word]), axis=0)
            labels = np.append(labels, np.array([[0,1,0]]), axis=0)
        else:
            sent_list = np.append(sent_list, np.array([new_word]), axis=0)
            labels = np.append(labels, np.array([[1,0,0]]), axis=0)

    input_data_ae.append(sent_list)
    op_labels_ae.append(labels)
input_data_ae = np.array(input_data_ae)
op_labels_ae  = np.array(op_labels_ae) 

In [35]:
input_data_ae = pad_sequences(input_data_ae,maxlen=30,dtype='float64',padding='post')
op_labels_ae = pad_sequences(op_labels_ae,maxlen=30,dtype='float64',padding='post') 

In [36]:
print(len(input_data_ae))
print(len(op_labels_ae)) 

4271
4271


In [39]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Input,LSTM,Embedding,Dropout,Activation,Bidirectional,TimeDistributed
from keras.layers import concatenate
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [40]:
x_train = input_data_ae[:4000]
x_test = input_data_ae[4000:]
y_train = op_labels_ae[:4000]
y_test = op_labels_ae[4000:] 

In [42]:
batch = 1

xin = Input(batch_shape=(batch,30,200),dtype='float')
seq = Bidirectional(LSTM(300,return_sequences=True),merge_mode='concat')(xin)
mlp1 = Dropout(0.2)(seq)
mlp2 = TimeDistributed(Dense(60,activation='softmax'))(mlp1)
mlp3 = Dropout(0.2)(mlp2)
mlp4 = TimeDistributed(Dense(3,activation='softmax'))(mlp3) 
model = Model(inputs=xin,outputs=mlp4)
model.compile(optimizer='Adam',loss='categorical_crossentropy')

In [43]:
model.fit(x_train,y_train,
         batch_size=batch,
         epochs=50,
         validation_data=(x_test,y_test)) 

Train on 4000 samples, validate on 271 samples
Epoch 1/50
4000/4000 [==============================] - 198s 50ms/step - loss: 0.1664 - val_loss: 0.0948
Epoch 2/50
4000/4000 [==============================] - 214s 53ms/step - loss: 0.1000 - val_loss: 0.0840
Epoch 3/50
4000/4000 [==============================] - 191s 48ms/step - loss: 0.0890 - val_loss: 0.0747
Epoch 4/50
4000/4000 [==============================] - 203s 51ms/step - loss: 0.0788 - val_loss: 0.0696
Epoch 5/50
4000/4000 [==============================] - 191s 48ms/step - loss: 0.0691 - val_loss: 0.0645
Epoch 6/50
4000/4000 [==============================] - 194s 48ms/step - loss: 0.0614 - val_loss: 0.0620
Epoch 7/50
4000/4000 [==============================] - 194s 49ms/step - loss: 0.0535 - val_loss: 0.0634
Epoch 8/50
4000/4000 [==============================] - 194s 48ms/step - loss: 0.0465 - val_loss: 0.0686
Epoch 9/50
4000/4000 [==============================] - 188s 47ms/step - loss: 0.0418 - val_loss: 0.0706
Epoch 10

In [53]:
val_pred = model.predict(x_test,batch_size=batch)
labels = []
for i in range(len(val_pred)):
    b = np.zeros_like(val_pred[i])
    b[np.arange(len(val_pred[i])),val_pred[i].argmax(1)] = 1
    labels.append(b)

In [54]:
print(val_pred.shape)

(271, 30, 3)


In [55]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [56]:
score = []
f1 = []
precision = []
recall = []
point = [] 

In [57]:
for i in range(len(y_test)):
    if (f1_score(labels[i],y_test[i],average='weighted') >.6):
        point.append(i)
    score.append(f1_score(labels[i],y_test[i],average='weighted'))
    precision.append(precision_score(labels[i],y_test[i],average='weighted'))
    recall.append(recall_score(labels[i],y_test[i],average='weighted'))

/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'r

In [58]:
print(len(point)/len(labels)*100) 

68.26568265682657


In [59]:
print(np.mean(score)) 

0.6873310922921418


In [60]:
print(np.mean(precision)) 

0.9741375645016355


In [61]:
print(np.mean(recall)) 

0.5769987699876998


In [62]:
print(score)
print("\n---------------x--------------\n")
print(precision)
print("\n---------------x--------------\n")
print(recall)  

[0.7125000000000001, 0.8235294117647058, 0.32727272727272727, 0.888888888888889, 0.4583333333333333, 0.6046511627906976, 0.888888888888889, 0.7144927536231884, 0.17413793103448275, 0.983050847457627, 0.8, 1.0, 0.7234042553191489, 0.4272727272727272, 0.8679245283018869, 0.7234042553191489, 0.5864864864864866, 0.9502824858757061, 0.8113378684807256, 0.6046511627906976, 0.8, 0.8976190476190476, 0.21269841269841272, 1.0, 0.4272727272727272, 0.9655172413793104, 0.7499999999999999, 0.3929824561403508, 0.7755102040816326, 0.418018018018018, 0.7891156462585034, 1.0, 0.6666666666666666, 0.4814814814814815, 0.17901234567901234, 0.7499999999999999, 0.4210526315789474, 0.4615384615384615, 0.6956521739130436, 0.5907859078590786, 0.888888888888889, 0.8679245283018869, 0.5864864864864866, 0.5714285714285715, 0.6666666666666666, 0.3, 0.5, 0.5714285714285715, 0.6666666666666666, 0.7234042553191489, 0.7755102040816326, 0.5904761904761905, 0.15777777777777777, 0.7499999999999999, 0.9285714285714286, 0.35